In [ ]:
import sys
import matplotlib.pyplot as plt
import pandas as pd
import scipy
from scipy import stats 
import numpy as np
import json
import statsmodels.stats.multitest as multi
import matplotlib.pyplot as plt
import math
sys.path.append('../scripts/')
import Docket_integration

In [ ]:
input_data = {"input_file_1":"../data/LUAD_GDSC_mut_matrix.csv",
"input_file_2":"../data/LUAD_GDSC_expr_mat.csv",
"output_file":"../Output/LUAD_GDSC_mut_expr_merge.csv",
}

In [ ]:
input_data2 = {"Mutlist": ["KRAS"],
"Exprlist": [],
"Output":"../Output/LUAD_GDSC_mut_exp_pair.csv",
}

In [ ]:
Mut_mat = pd.read_csv(input_data['input_file_1'], index_col = "Unnamed: 0")
drug_matrix = pd.read_csv(input_data['input_file_2'], index_col = "Unnamed: 0")

In [ ]:
label_for_merge = Docket_integration.matrix_comp(Mut_mat,drug_matrix )   

In [ ]:
if label_for_merge[0] > 0.9:
    Merged_mat = Docket_integration.merge_matrix(Mut_mat, drug_matrix, 'Row','Row')
elif label_for_merge[1] > 0.9:
    Merged_mat = Docket_integration.merge_matrix(Mut_mat, drug_matrix, 'Col','Col')
    
Merged_mat.to_csv(input_data['output_file'])

In [ ]:
if len(input_data2['Mutlist']) > 0:
    Features_groupA = input_data2['Mutlist']
else:
    Features_groupA = list(Mut_mat.columns.values)

if len(input_data2['Exprlist']) > 0:
    Features_groupB = input_data2['Exprlist']
else:
    Features_groupB = list(drug_matrix.columns.values)


result = Docket_integration.Integration_category_numeric(Merged_mat, Features_groupA, Features_groupB)

In [ ]:
result.to_csv(input_data2['Output'])

In [ ]:
result.sort_values(by = ['p'])

In [ ]:
result.loc[result['FDR'] < 0.05].sort_values(by = ['FDR'])